In [1]:
import leap
import cv2
import random
import os
import pickle
import sys
import time
import joblib

from leap import TrackingMode
import numpy as np
import tkinter as tk
from PIL import Image
from tkinter import filedialog, messagebox

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.models import Model, load_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

# from tensorflow.keras.regularizers import l1
# import psutil
# import subprocess

In [2]:

_TRACKING_MODES = {
    leap.TrackingMode.Desktop: "Desktop",
    leap.TrackingMode.HMD: "HMD",
    leap.TrackingMode.ScreenTop: "ScreenTop",
}
# edited
class GestureCapture:
    def __init__(self, required_frames=30, save_dir="gestures"):
        self.required_frames = required_frames
        self.frames = []
        self.capturing = False
        self.tracking_mode = leap.TrackingMode.Desktop
        self.save_dir = save_dir
        self.gesture_count = 0
        
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
    
    def start_capture(self):
        self.capturing = True
        self.frames = []  # Reset frames for new gesture
        print("Gesture capture started!")
    
    def stop_capture(self):
        self.capturing = False
        self.save_gesture()
        print("Gesture capture stopped!")
    
    def save_gesture(self):
        filename = os.path.join(self.save_dir, f"gesture_{self.gesture_count}.npy")
        np.save(filename, np.array(self.frames))
        print(f"Gesture {self.gesture_count} saved with {len(self.frames)} frames.")
        self.gesture_count += 1  # Increment for next gesture

    def process_frame(self, event):
        if len(event.hands) == 0:
            return None

        hand_data = []
        for hand in event.hands:
            for finger in hand.digits:
                for bone in finger.bones:
                    joint_position = [bone.prev_joint.x, bone.prev_joint.y, bone.prev_joint.z]
                    hand_data.append(joint_position)
        
        return np.array(hand_data).flatten()
    
    def render_visualization(self, event, canvas):
        canvas.output_image[:, :] = 0  # Clear previous frame
        
        if len(event.hands) == 0:
            cv2.putText(canvas.output_image, "No hands detected", (50, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
        else:
            for hand in event.hands:
                for finger in hand.digits:
                    for bone in finger.bones:
                        start = canvas.get_joint_position(bone.prev_joint)
                        end = canvas.get_joint_position(bone.next_joint)
                        if start and end:
                            cv2.line(canvas.output_image, start, end, (0, 255, 0), 2)
                            cv2.circle(canvas.output_image, start, 3, (255, 0, 0), -1)
                            cv2.circle(canvas.output_image, end, 3, (255, 0, 0), -1)

        cv2.putText(canvas.output_image, f"Frames: {len(self.frames)}", (50, 100),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        # cv2.imshow("Gesture Capture", canvas.output_image)

class TrackingListener(leap.Listener):
    def __init__(self, capture, canvas):
        self.capture = capture
        self.canvas = canvas

    def on_tracking_event(self, event):
        self.capture.render_visualization(event, self.canvas)
        
        if self.capture.capturing:
            frame_data = self.capture.process_frame(event)
            if frame_data is not None:
                self.capture.frames.append(frame_data)
            
            if len(self.capture.frames) >= self.capture.required_frames:
                self.capture.stop_capture()  # Stop and save gesture


class Canvas:
    # edited
    def __init__(self):
        self.screen_size = [500, 700]
        self.output_image = np.zeros((self.screen_size[0], self.screen_size[1], 3), np.uint8)


    # edited
    def get_joint_position(self, bone):
        if bone:
            return int(bone.x + (self.screen_size[1] / 2)), int(bone.z + (self.screen_size[0] / 2))
        else:
            return None

# edited
def main():
    canvas = Canvas()
    gesture_capture = GestureCapture(required_frames=100)
    tracking_listener = TrackingListener(gesture_capture, canvas)
    
    connection = leap.Connection()
    connection.add_listener(tracking_listener)

    with connection.open():
        print("  x: Exit")
        print("  s: Capture and save image")


        while True:
            cv2.imshow('Gesture Capturing', canvas.output_image)
            key = cv2.waitKey(1)

            if key == ord('s'):
                gesture_capture.start_capture()
            elif key == ord('x'):
                break
            
    cv2.destroyAllWindows()


class AppLock:
    def __init__(self):
        self.model_path_x = 'models/autoencoder_X.h5'
        self.model_path_y = 'models/autoencoder_Y.h5'
        self.model_path_z = 'models/autoencoder_Z.h5'
        
        self.autoencoder_X = None
        self.autoencoder_Y = None
        self.autoencoder_Z = None
        self.show_registration_window()

    def show_registration_window(self):
        """Show initial registration window"""
        self.root = tk.Tk()
        self.root.title("Gesture Password Registration")
        self.root.geometry("400x200")
        
        label = tk.Label(
            self.root, 
            text="Welcome!\nPlease register your password gesture.",
            font=('Arial', 12)
        )
        label.pack(pady=20)
        
        register_btn = tk.Button(
            self.root,
            text="Please capture the password gesture",
            command=self.register_password,
            font=('Arial', 11)
        )
        register_btn.pack(pady=20)
        
        self.root.mainloop()

    
    def load_and_process_gestures(self, gesture_dir):
        gesture_files = [os.path.join(gesture_dir, f) for f in os.listdir(gesture_dir) if f.endswith(".npy")]
        # Initialize lists for X, Y, Z channels
        X_data, Y_data, Z_data = [], [], []
        all_data = []

        # Load and preprocess each gesture
        for file in gesture_files:
            data = np.load(file)  # Load saved gesture data
            num_frames, num_features = data.shape  # Shape is (100, 60) -> (frames, features)
            
            if num_features % 3 != 0:
                raise ValueError(f"Unexpected shape {data.shape}, features should be a multiple of 3.")

            # Reshape into (frames, num_joints, 3)
            num_joints = num_features // 3
            data = data.reshape(num_frames, num_joints, 3)
            # Ensure it has the right shape (e.g., 100 frames, 21 joints)
            if data.shape != (100, 20, 3):  # Adjust if you expect a different shape
                print(f"Skipping gesture due to incorrect shape: {data.shape}")
                continue  # Skip malformed gesture files


            all_data.append(data)

            # Split channels
            X_data.append(data[:, :, 0])  # X coordinates
            Y_data.append(data[:, :, 1])  # Y coordinates
            Z_data.append(data[:, :, 2])  # Z coordinates


        return X_data, Y_data, Z_data, all_data
    
    # 2. Data Augmentation functions
    def add_jitter(self, data, sigma=0.01):
        return data + np.random.normal(0, sigma, data.shape)

    def scale_data(self, data, scale_range=(0.9, 1.1)):
        scale_factor = random.uniform(scale_range[0], scale_range[1])
        return data * scale_factor

    def rotate_data(self, data, angle_range=(-5, 5)):
        angle = np.radians(random.uniform(angle_range[0], angle_range[1]))
        cos_val, sin_val = np.cos(angle), np.sin(angle)
        rotation_matrix = np.array([[cos_val, -sin_val, 0], [sin_val, cos_val, 0], [0, 0, 1]])
        return np.dot(data.reshape(-1, 3), rotation_matrix).reshape(data.shape)

    def time_warp(self, data, factor_range=(0.8, 1.2)):
        """Apply time warping by interpolating along the time axis."""
        factor = random.uniform(factor_range[0], factor_range[1])
        new_length = max(1, int(data.shape[0] * factor))  # Adjust number of frames
        
        warped_data = np.zeros((new_length, data.shape[1], data.shape[2]))  # Same shape but different frames
        
        for j in range(data.shape[1]):  # Iterate over each joint
            for k in range(data.shape[2]):  # Iterate over X, Y, Z
                warped_data[:, j, k] = np.interp(
                    np.linspace(0, data.shape[0] - 1, new_length),  # New frame indices
                    np.arange(data.shape[0]),  # Original frame indices
                    data[:, j, k]  # Original joint data
                )
        
        return warped_data

    def resize_sequences(self, data_list, target_length=100):
        """Resize sequences to a fixed number of frames."""
        resized_data = []
        
        for seq in data_list:
            resized_seq = np.zeros((target_length, seq.shape[1], seq.shape[2]))
            for j in range(seq.shape[1]):  # Iterate over joints
                for k in range(seq.shape[2]):  # Iterate over X, Y, Z
                    resized_seq[:, j, k] = np.interp(
                        np.linspace(0, seq.shape[0] - 1, target_length),
                        np.arange(seq.shape[0]),
                        seq[:, j, k]
                    )
            resized_data.append(resized_seq)
        
        return np.array(resized_data)

    def ensure_3d(self, data_list):
        """Ensure all sequences are in (frames, num_joints, 3) format."""
        corrected_data = []
        
        for seq in data_list:
            seq = np.array(seq)
            if len(seq.shape) == 2:  # If (frames, num_joints), add a last dimension (Z=1)
                seq = seq.reshape(seq.shape[0], seq.shape[1], 1)  # Convert to (frames, num_joints, 1)
            corrected_data.append(seq)

        return corrected_data

    def augment_data(self, data):
        augmented_X = []
        augmented_Y = []
        augmented_Z = []

        # Original data
        augmented_X.append(data[:, :, 0])
        augmented_Y.append(data[:, :, 1])
        augmented_Z.append(data[:, :, 2])

        # Augmented versions
        for _ in range(3):  # Generate 3 augmented samples per gesture
            aug_data = self.add_jitter(data)
            aug_data = self.scale_data(aug_data)
            aug_data = self.rotate_data(aug_data)
            aug_data = self.time_warp(aug_data)
            
            augmented_X.append(aug_data[:, :, 0])
            augmented_Y.append(aug_data[:, :, 1])
            augmented_Z.append(aug_data[:, :, 2])

        augmented_X = self.ensure_3d(augmented_X)
        augmented_Y = self.ensure_3d(augmented_Y)
        augmented_Z = self.ensure_3d(augmented_Z)


        # Apply resizing
        augmented_X = self.resize_sequences(augmented_X, target_length=100)
        augmented_Y = self.resize_sequences(augmented_Y, target_length=100)
        augmented_Z = self.resize_sequences(augmented_Z, target_length=100)

        # Convert lists to numpy arrays
        augmented_X = np.array(augmented_X)
        augmented_Y = np.array(augmented_Y)
        augmented_Z = np.array(augmented_Z)

        return augmented_X, augmented_Y, augmented_Z

    def build_autoencoder(self, input_dim, encoding_dim=64):
        input_layer = Input(shape=(input_dim,))
        # Encoder
        encoded = Dense(128, activation=None)(input_layer)
        encoded = LeakyReLU(alpha=0.1)(encoded)
        encoded = BatchNormalization()(encoded)
        encoded = Dropout(0.2)(encoded)
        
        encoded = Dense(encoding_dim, activation=None)(encoded)
        encoded = LeakyReLU(alpha=0.1)(encoded)
        
        # Decoder
        decoded = Dense(128, activation=None)(encoded)
        decoded = LeakyReLU(alpha=0.1)(decoded)
        decoded = BatchNormalization()(decoded)
        
        decoded = Dense(input_dim, activation="sigmoid")(decoded)  # Output should match input shape
        
        autoencoder = Model(input_layer, decoded)

        autoencoder.compile(optimizer="adam", loss="mse")  # Use the registered function
        return autoencoder
    
    
    def train_autoencoder(self, X_data, Y_data, Z_data, batch_size=16, epochs=100):
        # Load preprocessed data
        X_data = np.load("preprocessed/X_data.npy")
        Y_data = np.load("preprocessed/Y_data.npy")
        Z_data = np.load("preprocessed/Z_data.npy")

        # Get input dimensions
        input_dim = X_data.shape[1] * X_data.shape[2]  # frames * joints

        # Flatten the data for training
        X_train = X_data.reshape(X_data.shape[0], input_dim)
        Y_train = Y_data.reshape(Y_data.shape[0], input_dim)
        Z_train = Z_data.reshape(Z_data.shape[0], input_dim)

        """Train autoencoder on image data"""

        if self.autoencoder_X is None and self.autoencoder_Y is None and self.autoencoder_Z is None:
            self.autoencoder_X = self.build_autoencoder(input_dim)
            self.autoencoder_Y = self.build_autoencoder(input_dim)
            self.autoencoder_Z = self.build_autoencoder(input_dim)
        
        # Train the model
        self.autoencoder_X.fit(X_train, X_train, epochs=epochs, batch_size=batch_size, shuffle = True, verbose=1)
        self.autoencoder_Y.fit(Y_train, Y_train, epochs=epochs, batch_size=batch_size, shuffle = True, verbose=1)
        self.autoencoder_Z.fit(Z_train, Z_train, epochs=epochs, batch_size=batch_size, shuffle = True, verbose=1)

        # Save trained models
        self.autoencoder_X.save(self.model_path_x)
        self.autoencoder_Y.save(self.model_path_y)
        self.autoencoder_Z.save(self.model_path_z)

        print("Autoencoder training completed! Models saved.")

    # def authenticate(self, autoencoder, scaler, test_data):
    #     test_data = scaler.transform(test_data.reshape(1, -1))
    #     reconstructed = autoencoder.predict(test_data)
    #     error = np.mean(np.abs(test_data - reconstructed))
    #     return error
    
    
    
    def get_threshold(self,gestures_dir):
        autoencoder_x = load_model("models/autoencoder_X.h5", custom_objects={"mse": tf.keras.losses.MeanSquaredError()})
        autoencoder_y = load_model("models/autoencoder_Y.h5", custom_objects={"mse": tf.keras.losses.MeanSquaredError()})
        autoencoder_z = load_model("models/autoencoder_Z.h5", custom_objects={"mse": tf.keras.losses.MeanSquaredError()})

        # Load saved scalers
        scaler_x = joblib.load("models/scaler_x.pkl")
        scaler_y = joblib.load("models/scaler_y.pkl")
        scaler_z = joblib.load("models/scaler_z.pkl")


        gesture_files = [os.path.join(gestures_dir, f) for f in os.listdir(gestures_dir) if f.endswith(".npy")]
        # List to store combined reconstruction errors
        combined_errors = []

        # Loop through all gestures
        for file in gesture_files:
            # Load gesture data
            gesture_data = np.load(file)  # Shape: (100, 20, 3)
            num_frames, num_features = gesture_data.shape
            # Reshape and split into channels
            num_joints = num_features // 3
            gesture_data = gesture_data.reshape(num_frames, num_joints, 3)

            # Split data into X, Y, and Z channels
            X_test = gesture_data[:, :, 0]  # Shape: (100, 20)
            Y_test = gesture_data[:, :, 1]
            Z_test = gesture_data[:, :, 2]


            X_test = X_test.reshape(-1, 1)  # Ensure shape matches what scaler was trained on
            Y_test = Y_test.reshape(-1, 1)
            Z_test = Z_test.reshape(-1, 1)

            X_test = scaler_x.transform(X_test).reshape(100, 20)  # Reshape back after scaling
            Y_test = scaler_y.transform(Y_test).reshape(100, 20)
            Z_test = scaler_z.transform(Z_test).reshape(100, 20)


            input_dim = X_test.shape[0] * X_test.shape[1]  # 100 * 20 = 600
            X_test = X_test.reshape(1, input_dim)  # Reshape to (1, 600)
            Y_test = Y_test.reshape(1, input_dim)
            Z_test = Z_test.reshape(1, input_dim)

            # Get reconstructed data from each autoencoder
            X_reconstructed = autoencoder_x.predict(X_test)
            Y_reconstructed = autoencoder_y.predict(Y_test)
            Z_reconstructed = autoencoder_z.predict(Z_test)

            # Compute reconstruction errors (MSE) for each channel
            error_x = mean_squared_error(X_test.flatten(), X_reconstructed.flatten())
            error_y = mean_squared_error(Y_test.flatten(), Y_reconstructed.flatten())
            error_z = mean_squared_error(Z_test.flatten(), Z_reconstructed.flatten())

            # Compute final combined error (can be sum, mean, or weighted)
            combined_error = (error_x + error_y + error_z) / 3  # Taking the average error
            combined_errors.append(combined_error)

        # Compute mean (μ) and standard deviation (σ) of combined errors
        mu = np.mean(combined_errors)
        sigma = np.std(combined_errors)

        # Set authentication threshold
        k = 0.25  # Adjust if needed
        threshold = mu + 2 * sigma

        with open('threshold.pkl','wb') as f:
            pickle.dump(threshold,f)

        # Print threshold details
    
        print(f"Number of gestures processed: {len(combined_errors)}")
        print(f"Mean Reconstruction Error: {mu:.5f}")
        print(f"Standard Deviation: {sigma:.5f}")
        print(f"Threshold for Authentication: {threshold:.5f}")

        return threshold

    
    def register_password(self):
        """Register the password gesture and train autoencoder"""
        main()

        folder_path = 'gestures'
        try:
            status_label = tk.Label(self.root, text="Training model...", font=('Arial', 10))
            status_label.pack(pady=10)
            self.root.update()

            X_data_list, Y_data_list, Z_data_list, all_data_list = self.load_and_process_gestures(folder_path)
            final_X_data, final_Y_data, final_Z_data = [], [], []

            for gesture_data in all_data_list:
                aug_x, aug_y, aug_z = self.augment_data(gesture_data)
                final_X_data.extend(aug_x)
                final_Y_data.extend(aug_y)
                final_Z_data.extend(aug_z)

            final_X_data = np.array(final_X_data)
            final_Y_data = np.array(final_Y_data)
            final_Z_data = np.array(final_Z_data)

            # Normalize the entire training set
            scaler_x, scaler_y, scaler_z = MinMaxScaler(), MinMaxScaler(), MinMaxScaler()
            final_X_data = scaler_x.fit_transform(final_X_data.reshape(-1, final_X_data.shape[-1])).reshape(final_X_data.shape)
            final_Y_data = scaler_y.fit_transform(final_Y_data.reshape(-1, final_Y_data.shape[-1])).reshape(final_Y_data.shape)
            final_Z_data = scaler_z.fit_transform(final_Z_data.reshape(-1, final_Z_data.shape[-1])).reshape(final_Z_data.shape)

            joblib.dump(scaler_x, "models/scaler_x.pkl")
            joblib.dump(scaler_y, "models/scaler_y.pkl")
            joblib.dump(scaler_z, "models/scaler_z.pkl")

            np.save("preprocessed/X_data.npy", final_X_data)
            np.save("preprocessed/Y_data.npy", final_Y_data)
            np.save("preprocessed/Z_data.npy", final_Z_data)

            
            print("Training autoencoder...")
            self.train_autoencoder(final_X_data, final_Y_data, final_Z_data)
            self.get_threshold(folder_path)
            
            messagebox.showinfo("Success", "Password gesture registered successfully!")
            print("Registration complete!")

            self.root.destroy()
            
            self.root = tk.Tk()
            self.root.withdraw()
            sys.exit()
            # self.monitor_todo_app()
            
        except Exception as e:
            messagebox.showerror("Error", f"Registration failed: {str(e)}")
            print(f"Registration failed: {e}")



In [ ]:
def AppLock_main():
    app_lock = AppLock()

if __name__ == "__main__":
    AppLock_main()

  x: Exit
  s: Capture and save image
